<a href="https://colab.research.google.com/github/ashikita/openalex-api-notebook/blob/main/cited_by_count_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pyalex
from pyalex import Works
import csv
pyalex.config.email = "hogehoge@m.kyushu-u.ac.jp"

# 論文データを取得（最大10件）
results = Works() \
  .filter(
      authorships={"institutions": {"ror": "00p4k0j84"}},
      from_publication_date="2024-01-01",
      to_publication_date="2024-12-31"
  ) \
  .sort(cited_by_count="desc") \
  .get(per_page=100)

# 必要な要素だけ抽出
filtered_results = []
for work in results:
    corresponding_authors = []
    for authorship in work.get("authorships", []):
        if authorship.get("is_corresponding"):
            for institution in authorship.get("institutions", []):
                if institution.get("ror") == "https://ror.org/00p4k0j84":
                    display_name = authorship.get("author", {}).get("display_name")
                    if display_name:
                        corresponding_authors.append(display_name)
                    break

    filtered_results.append({
        "doi": work.get("doi"),
        "publication_date": work.get("publication_date"),
        "cited_by_count": work.get("cited_by_count"),
        "is_oa": work.get("open_access", {}).get("is_oa"),
        "oa_status": work.get("open_access", {}).get("oa_status"),
        "oa_url": work.get("open_access", {}).get("oa_url"),
        "any_repository_has_fulltext": work.get("open_access", {}).get("any_repository_has_fulltext"),
        "corresponding_authors": ", ".join(corresponding_authors)
    })

# TSVファイルに保存
with open("output.tsv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "doi", "publication_date", "cited_by_count",
        "is_oa", "oa_status", "oa_url", "any_repository_has_fulltext",
        "corresponding_authors"
    ], delimiter='\t')
    writer.writeheader()
    writer.writerows(filtered_results)

print("✅ データは output.tsv に保存されました。")